In [47]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader , Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torchvision

import nevergrad as ng

import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import copy
import os

import sys
sys.path.append("../")
sys.path.append("nevergrad/")



import models
from models import CNN_Simple as CNN_Simple
from models import All_CNN_C as All_CNN_C
from models import VAE as VAE
from models import LSTM as LSTM

In [48]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

P1

In [49]:
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Lambda(lambda x: x.double()),
    transforms.Normalize((0.5,), (0.5,))])
training_set = torchvision.datasets.FashionMNIST("./data", train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST("./data", train=False, transform=transform, download=True)
training_loader = torch.utils.data.DataLoader(training_set, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=64, shuffle=False)


In [50]:
def load_params(model, param_tensor):
    current_index = 0
    for param in model.parameters():
        param_length = param.numel()
        #print(param_length, param.size())
        param.data = param_tensor[current_index:current_index + param_length].reshape(param.size())
        current_index += param_length

In [51]:
class TrainManager:
    def __init__(self,model,dataloader_train,dataloader_test,loss,output):
        self.model = model
        self.best_model = None
        self.dataloader_train = dataloader_train
        self.dataloader_test = dataloader_test
        self.nb_batch = len(dataloader_train)
        self.best_score = 1e9
        self.loss = loss
        self.iteration = 0
        self.output = output
        self.best_output = output
    
    def weights_updating(self,weights):
        for n, layer in enumerate(self.model.parameters()):
            layer.data = torch.from_numpy(weights[n]).to(dtype=torch.double)
            
    def evaluate(self):
        total_loss = 0
        for x , y in self.dataloader_test :
            yhat = self.model(x)
            loss = self.loss(yhat,y)
            total_loss += loss.item()
        total_loss /= len(self.dataloader_test)
        return total_loss

    def cost_function(self, parameters):
        load_params(self.model, torch.tensor(parameters, dtype=torch.double))

        test_loss = self.evaluate()
        if self.best_score > test_loss :
            self.best_score = test_loss
            self.best_model = copy.deepcopy(self.model)

        print(f'test loss function (cross entropy loss) : {test_loss}, best score : {self.best_score}')
        self.output[self.iteration,:] = np.array([test_loss])
        self.best_output[self.iteration,:] = np.array([self.best_score])
        self.iteration += 1
        return test_loss

In [52]:
model = CNN_Simple().to(device)
loss = nn.CrossEntropyLoss()
model.load_state_dict(torch.load('models/CNN_Simple.pt'))


<All keys matched successfully>

In [53]:
epochs = 20

output = np.empty((epochs,2))

trainer = TrainManager(model,training_loader,validation_loader,loss,output)
fitness = trainer.cost_function

# Compute number of parameters of the model + initialize parametrization
num_params = sum(p.numel() for p in model.parameters())
parametrization = ng.p.Array(shape=(num_params,))
optimizer = ng.optimizers.SPSA(parametrization=parametrization, budget=epochs)



In [54]:
learned_param = optimizer.minimize(fitness)

test loss function (cross entropy loss) : 2.4347994006532736, best score : 2.4347994006532736
test loss function (cross entropy loss) : 2.369026054651724, best score : 2.369026054651724
test loss function (cross entropy loss) : 2.308682360415839, best score : 2.308682360415839
test loss function (cross entropy loss) : 2.351624552889627, best score : 2.308682360415839


P2

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Lambda(lambda x: x.double())])

In [ ]:
training_set = torchvision.datasets.MNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.MNIST('./data', train=False, transform=transform, download=True)

In [ ]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=64, shuffle=False)

In [ ]:
model = models.VAE()

In [ ]:
torch.save(model.state_dict(), 'models/VAE.pt')